In [ ]:
from faker import Faker
from collections import ChainMap
import random
import pandas as pd
import time
from kafka import *
from confluent_kafka.admin import *
import json 
from confluent_kafka import *

In [ ]:
faker = Faker()

#### Customer Data 
- customer_id
- customer_fname
- customer_lname
- gender
- email
- country
- device_type
- age
  
-------------------

#### Product Data
- product_id
- product_name
- price
- category
  
-------------------

#### Transaction Data
- transaction_id
- Customer Data
- Product Data
- timestamp
- paymnet_method
- status
- quantity
- total_amount

In [ ]:
def get_customer_data():
    customer_id = random.randint(1,100000)
    
    genders = ["male" ,"female"]
    gender = random.choice(genders)
    
    if gender == "male":
        customer_gender = "Male"
        customer_first_name = faker.first_name_male()
        customer_last_name = faker.last_name_male()
    else:
        customer_gender = "Female"
        customer_first_name = faker.first_name_female()
        customer_last_name = faker.last_name_female()
    customer_country = faker.country()
    
    devices = ["Mobile" ,"PC" ,"Laptop" ,"Tablet" ,"Smart TV"]
    device_type = random.choice(devices)
    customer_age = random.randint(7,100)
    
    emails = ["gmail" ,"yahoo" ,"outlook" ,"zoho"]
    email = random.choice(emails)
    customer_email = customer_first_name + customer_last_name +"@"+ email + ".com"
    
    customer_data = {
        "customer_id" : customer_id,
        "customer_first_name" : customer_first_name,
        "customer_last_name" : customer_last_name,
        "customer_gender" : customer_gender,
        "customer_country" : customer_country,
        "customer_email" : customer_email,
        "device_type" : device_type,
        "customer_age" : customer_age
    }
    return customer_data

In [ ]:
categories = {
    "Electronics": [
        ["Laptop", 899.99], ["Smartphone", 599.99], ["Tablet", 299.99], 
        ["Smartwatch", 199.99], ["Headphones", 79.99], ["Bluetooth Speaker", 49.99], 
        ["Camera", 499.99], ["Gaming Console", 399.99], ["Monitor", 179.99], 
        ["Keyboard", 49.99], ["Drone", 249.99], ["Power Bank", 29.99], 
        ["External Hard Drive", 99.99], ["Smart TV", 699.99], ["Router", 129.99]
    ],
    "Clothing": [
        ["T-shirt", 19.99], ["Jeans", 39.99], ["Jacket", 59.99], 
        ["Sneakers", 49.99], ["Dress", 29.99], ["Sweater", 24.99], 
        ["Shorts", 19.99], ["Cap", 14.99], ["Scarf", 9.99], 
        ["Socks", 4.99], ["Blazer", 89.99], ["Sportswear", 39.99], 
        ["Raincoat", 49.99], ["Formal Shoes", 59.99], ["Belt", 19.99]
    ],
    "Books": [
        ["Fiction Novel", 14.99], ["Cookbook", 22.99], ["Biography", 18.99], 
        ["Mystery Thriller", 9.99], ["Science Textbook", 49.99], 
        ["Fantasy Novel", 12.99], ["Historical Fiction", 16.99], 
        ["Self-Help Guide", 13.99], ["Children's Book", 6.99], ["Graphic Novel", 19.99], 
        ["Poetry Collection", 11.99], ["Travel Guide", 17.99], ["Romance Novel", 8.99], 
        ["Horror Story", 10.99], ["Programming Book", 29.99]
    ],
    "Toys": [
        ["Action Figure", 12.99], ["Puzzle", 9.99], ["Board Game", 24.99], 
        ["Doll", 14.99], ["Lego Set", 49.99], ["Stuffed Animal", 10.99], 
        ["Remote Control Car", 19.99], ["Rubik's Cube", 5.99], 
        ["Building Blocks", 14.99], ["Yo-Yo", 3.99], ["Water Gun", 7.99], 
        ["Kite", 8.99], ["Toy Train", 15.99], ["Marbles", 2.99], ["Play-Doh", 4.99]
    ],
    "Groceries": [
        ["Milk", 1.99], ["Eggs", 2.49], ["Cheese", 4.99], ["Bread", 2.29], 
        ["Coffee", 8.99], ["Tea", 3.99], ["Rice", 1.99], ["Pasta", 2.49], 
        ["Cereal", 3.49], ["Butter", 3.79], ["Juice", 2.99], ["Sugar", 1.99], 
        ["Cooking Oil", 5.99], ["Flour", 1.49], ["Salt", 0.99], 
        ["Tomatoes", 2.49], ["Potatoes", 1.19], ["Onions", 1.29], 
        ["Garlic", 0.99], ["Bananas", 0.59]
    ],
    "Furniture": [
        ["Chair", 49.99], ["Table", 79.99], ["Sofa", 299.99], ["Bed", 199.99], 
        ["Wardrobe", 159.99], ["Bookshelf", 99.99], ["Dining Set", 249.99], 
        ["TV Stand", 129.99], ["Desk", 129.99], ["Recliner", 199.99], 
        ["Stool", 39.99], ["Cupboard", 89.99], ["Coffee Table", 59.99], 
        ["Bench", 79.99], ["Side Table", 29.99]
    ],
    "Home Appliances": [
        ["Washing Machine", 399.99], ["Refrigerator", 799.99], ["Microwave", 129.99], 
        ["Blender", 49.99], ["Vacuum Cleaner", 99.99], ["Iron", 29.99], 
        ["Toaster", 19.99], ["Air Conditioner", 299.99], ["Heater", 89.99], 
        ["Water Purifier", 149.99], ["Dishwasher", 349.99], ["Ceiling Fan", 39.99], 
        ["Electric Kettle", 24.99], ["Oven", 199.99], ["Juicer", 49.99]
    ],
    "Sports Equipment": [
        ["Football", 19.99], ["Basketball", 29.99], ["Tennis Racket", 39.99], 
        ["Badminton Racket", 19.99], ["Cricket Bat", 29.99], ["Yoga Mat", 19.99], 
        ["Dumbbells", 29.99], ["Treadmill", 499.99], ["Skipping Rope", 9.99], 
        ["Hockey Stick", 39.99], ["Cycling Helmet", 29.99], ["Swim Goggles", 14.99], 
        ["Fishing Rod", 59.99], ["Golf Club", 99.99], ["Bowling Ball", 39.99]
    ]
}


In [ ]:
def get_product_data():
    product_id = random.randint(1,1000)
    category = random.choice(list(categories.keys()))
    product_name = random.choice(categories[category])[0]
    product_price = random.choice(categories[category])[1]

    product_data = {
        "product_id" : product_id,
        "category" : category,
        "product_name" : product_name,
        "product_price" : product_price
    }

    return product_data

In [ ]:
def get_transaction_data():
    
    customer_data = get_customer_data()
    product_data = get_product_data()
    transaction_id = faker.uuid4()
    timestamp = faker.date_time_this_year().isoformat()
    payment_methods = ["Credit Cards" ,"Debit Cards" ,"Cash" ,"Bank Transfers" ,"PayPal" ,"Mobile Wallets"]
    payment_method = random.choice(payment_methods)
    quantity = random.randint(1 ,25)
    total_amount = round(product_data["product_price"] * quantity ,2)
    status = random.choice(["Completed", "Pending", "Canceled"])

    transaction_data = {
        "transaction_id" : transaction_id ,
        "customer_data" : customer_data ,
        "product_data" : product_data ,
        "timestamp" : timestamp ,
        "payment_method" : payment_method ,
        "quantity" : quantity ,
        "total_amount" : total_amount ,
        "status" : status
    }

    return transaction_data

In [ ]:
import logging

logging.getLogger('kafka').setLevel(logging.WARNING)

kafka_producer = KafkaProducer(
    bootstrap_servers=['host.docker.internal:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

topic_name = "real-time-pipeline"

In [ ]:
while True:
    try:
        
        transactional_data = get_transaction_data()
        kafka_producer.send(topic_name ,transactional_data)
        print("Data :" ,transactional_data)
    
    except Exception as error:
        print(f"There is an Error and it is {error}")

    time.sleep(5)


# kafka_producer.flush()